In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from tqdm import tqdm
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import tensorflow_text
import spacy
import numpy

In [5]:
df = pd.read_csv("../Datasets/train-200.csv",index_col=[0])
#df=df.sample(n=10000)
#df=df.drop(["Unnamed: 0"],axis=1)


/home/ciprian/.local/lib/python3.9/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
df.head()

,sentiment,text
0,2,this sound track was it paints the in your min...
1,2,reading a lot of reviews saying that this is t...
2,2,this soundtrack is my favorite music of all ha...
3,2,i truly like this soundtrack and i enjoy video...
4,2,if played the you know how divine the music ev...


In [4]:
df.to_csv("../Datasets/train-200.csv")

In [7]:
texts=df["text"]
nlp = spacy.load("en_core_web_sm")

In [9]:
stopwords=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y']
stopwords_neg=['no', 'nor', 'not', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't" , "don", "that'll"]
print(stopwords)
print(stopwords_neg)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', '

In [ ]:
tokenized_texts=[]
for text in texts:
    doc=nlp(text)
    filtered_tokens = [token for token in doc if not stopwords]
    tokenized_texts.append(filtered_tokens)

In [ ]:
max=0
for text in tokenized_texts:
    if len(text)>max:
        max=len(text)
print(max)

In [ ]:
max=256
expanded_tokenized_texts=[]
for text in tokenized_texts:
    text=text[:max] + [nlp("null")]*(max - len(text))
    expanded_tokenized_texts.append(text)

In [ ]:
normalized_texts=[]
for text in  expanded_tokenized_texts:
    vectors=[]
    for word in text:
        vectors.append((numpy.array(word.vector)))
    normalized_texts.append(numpy.array(vectors))

In [ ]:
print(numpy.array(normalized_texts).shape)



In [ ]:
from sklearn.preprocessing import OneHotEncoder

type_one_hot = OneHotEncoder(sparse=False).fit_transform(
  df.sentiment.to_numpy().reshape(-1, 1)
)

In [ ]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
train_news, test_news, y_train, y_test =\
  train_test_split(
    normalized_texts, 
    type_one_hot, 
    test_size=.1, 
    random_state=RANDOM_SEED
  )

In [ ]:
train_news=np.array(train_news)
test_news=np.array(test_news)
y_train=np.array(y_train)
y_test=np.array(y_test)


In [ ]:
print(train_news[1].shape)
print(test_news.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
print(train_news.shape)

In [ ]:
model = keras.Sequential()

model.add(keras.layers.cell(units, return_sequences=False))

model.add(
  keras.layers.Dense(
    units=128,
    input_shape=(train_news[1].shape[0],train_news[1].shape[1]),
    activation='relu'
  )
)
model.add(
  keras.layers.Dropout(rate=0.5)
)

model.add(keras.layers.GlobalMaxPool1D())

model.add(
  keras.layers.Dense(
    units=32,
    activation='relu'
  )
)
model.add(
  keras.layers.Dropout(rate=0.5)
)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(2, activation='softmax'))
model.compile(
    loss='categorical_crossentropy', 
    optimizer=keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_news, y_train, 
    epochs=30, 
    batch_size=32, 
    validation_split=0.1,
    verbose=1, 
    shuffle=True
)

In [ ]:
plt.plot(history.history['loss'], label='train loss'),
plt.plot(history.history['val_loss'], label='val loss'),
plt.xlabel("epoch")
plt.ylabel("Cross-entropy loss"),

In [ ]:
    plt.plot(history.history['accuracy'], label='train accuracy'),
    plt.plot(history.history['val_accuracy'], label='val accuracy'),
    plt.xlabel("epoch"),
    plt.ylabel("accuracy"),
    plt.legend();

In [ ]:
model.evaluate(test_news, y_test)

In [ ]:
model.save('./Model/')